In [5]:
import psycopg2
import csv
from faker import Faker
import random
from datetime import datetime, timedelta
from decimal import Decimal
fake = Faker()

In [6]:
def get_connection():
    return psycopg2.connect(
        host="localhost",
        database="5310 project",
        user="postgres",
        password="123"
    )


In [7]:
def generate_customer_orders(n=5000):
    conn = get_connection()
    cursor = conn.cursor()
    
    # Fetch product and customer details
    cursor.execute("SELECT product_id, sell_price FROM products")
    products = cursor.fetchall()
    cursor.execute("SELECT customer_id FROM customers")
    customers = cursor.fetchall()
    
    orders = []
    
    for _ in range(n):
        product_id, sell_price = random.choice(products)
        customer_id = random.choice(customers)[0]
        quantity = random.randint(1, 10)
        
        # Generate a random date and time
        days_ago = random.randint(0, 365)
        hours_ago = random.randint(0, 23)
        minutes_ago = random.randint(0, 59)
        seconds_ago = random.randint(0, 59)
        
        order_date = datetime.now() - timedelta(days=days_ago, hours=hours_ago, minutes=minutes_ago, seconds=seconds_ago)
        order_date_only = order_date.date()
        
        # Fetch discount percentage for the product and check if the discount is valid for the order date
        cursor.execute("SELECT discount_percentage, starting_time, ending_time FROM discounts WHERE product_id = %s", (product_id,))
        discount_result = cursor.fetchone()
        
        if discount_result:
            discount_percentage, starting_time, ending_time = discount_result
            discount_percentage = Decimal(discount_percentage)
            
            # Convert the starting_time and ending_time to date only
            starting_date_only = starting_time.date()
            ending_date_only = ending_time.date()
            
            # Check if the order_date is within the discount period
            if not (starting_date_only <= order_date_only <= ending_date_only):
                discount_percentage = Decimal('0.0')
        else:
            discount_percentage = Decimal('0.0')
        
        # Fetch coupon amount for the customer
        cursor.execute("SELECT coupon_amount FROM coupons WHERE customer_id = %s", (customer_id,))
        coupon_result = cursor.fetchone()
        coupon_amount = Decimal(coupon_result[0]) if coupon_result else Decimal('0.0')
        
        # Calculate the price considering discount and coupon
        price_before_coupon = sell_price * (Decimal('1.0') - discount_percentage / Decimal('100.0')) * quantity
        price = max(Decimal('0.0'), price_before_coupon - coupon_amount)
        
        try:
            cursor.execute(
                "INSERT INTO customer_order (product_id, quantity, price, date, customer_id) VALUES (%s, %s, %s, %s, %s)",
                (product_id, quantity, price, order_date, customer_id)
            )
        except psycopg2.IntegrityError as e:
            print(f"Error inserting data: {e}")
            conn.rollback()  # Rollback in case of error
            continue
        
        orders.append((product_id, quantity, price, order_date, customer_id))

    conn.commit()

    # Write the generated order data to a CSV file
    with open('C:/Users/Yihua/Desktop/5310 project/customer_orders.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['product_id', 'quantity', 'price', 'date', 'customer_id'])
        for order in orders:
            csv_writer.writerow([
                order[0],  # product_id
                order[1],  # quantity
                order[2],  # total_price
                order[3].strftime('%Y-%m-%d %H:%M:%S'),  # order_date
                order[4]   # customer_id
            ])

if __name__ == "__main__":
    generate_customer_orders()